# Image Stacking

https://forofotografiasalva.blogspot.com/2014/12/focus-stacking-o-enfoque-de-apilamiento.html

El apilamiento de imágenes (image stacking) con python es rápido, eficiente y permite un filtrado personalizado durante el proceso de apilado. ¿Por qué querrías apilar imágenes? Bueno, permite "exposiciones largas manuales" ya que, al dejar que el obturador de la cámara permanezca abierto durante largos períodos de tiempo, puede tomar varias fotos en serie y apilar las imágenes juntas. La otra ventaja es que puede aplicar diferentes técnicas de apilamiento, como resaltar los puntos brillantes en una imagen para producir rastros de estrellas.

Entonces, ¿cómo hacemos esto en python? Esencialmente, la forma más fácil de apilar es simplemente sumar todos los valores de píxeles para todas las imágenes deseadas, y dividir por el número de imágenes, creando una imagen promedio. Sobre la base de nuestro conocimiento de PIL, podemos escribir un script a partir de la importación de las bibliotecas necesarias:

### Introducción a PIL:

http://onlyjus-photopy.blogspot.com/2012/09/pil-introduction-batch-image-converting.html

Para abrir una imagen usando PIL, usamos el módulo de Imagen de PIL (en una shell interactiva o script de python):

In [ ]:
from PIL import Image
im=Image.open('./example.JPG')

Esto crea una instancia PIL de la imagen especificada, im.

Ahora podemos usar esta instancia para realizar una variedad de tareas, como mostrar la imagen:

In [ ]:
im.show()

Girando la imagen en sentido contrario a las agujas del reloj en un ángulo especificado en grados, creando una nueva instancia de imagen de la imagen girada:

In [ ]:
im2=im.rotate(45)

Algunos de los atributos de clase serían:

In [ ]:
>>> im.size
(3872, 2592)
>>> im.format
'JPEG'
>>> im.mode
'RGB'

Cambiar el tamaño de la imagen, preservando la relación de aspecto:

In [ ]:
im.thumbnail([128,128])

Esto redimensionará la dimensión máxima, conservando la relación de aspecto. Para controlar el esquema de interpolación del cambio de tamaño, puede pasar un argumento de filtro como NEAREST, BILINEAR, BICUBIC o ANTIALIAS (la mejor calidad):

In [ ]:
im.thumbnail([128,128],Image.ANTIALIAS)

Cambiar el tamaño de la imagen, sin conservar la relación de aspecto:

In [ ]:
im.resize([128,128])

Recortar la imagen proporcionando un cuadro de recorte en términos de: Izquierda, Superior, Derecha, Inferior; en términos de píxeles, esto también devuelve una nueva instancia de imagen:

In [ ]:
im3=im.crop([10,40,400,350])

Finalmente, guardando la imagen resultante como jpg (indicado por la extensión de archivo):

In [ ]:
im.save('./exampleOut.jpg')

También puede indicar explícitamente a la función en qué formato guardarlo (en su caso, como imagen png):

In [ ]:
im.save('./exampleOut', 'PNG')
#This will not automatically append the file extension to the file name.

### Image Stacking: Averaging (promediando)

https://onlyjus-photopy.blogspot.com/2012/09/image-stacking.html

In [ ]:
from PIL import Image
import glob
import numpy as np

Se buscan todas las imágenes en una carpeta, en este caso los archivos * .png:

In [ ]:
imgList = glob.glob('./*.png')

A continuación, creamos un bucle para recorrer todas las imágenes que encontramos, sin embargo, necesitamos saber si es la primera imagen para inicializar la variable de imagen sumada:

In [ ]:
first = True
for img in imgList:

Ahora, para la parte difícil. Observe cómo importamos numpy. El problema con el uso de la clase de imagen PIL es que el tipo de datos para los valores de píxeles RGB es unit8, o un valor entre 0 y 255. Si el valor resultante excede 255, entonces se reinicia desde 0 (es decir, 140 + 210 = 94). Esto se conoce como un desbordamiento y cambiará el color del píxel. Para superar este problema, convertiremos la imagen PIL en una matriz numpy:

In [ ]:
    temp = np.asarray(Image.open(img))

Luego, cambie el tipo de datos de la matriz. Existen varias opciones para los tipos de datos. Vamos a tratar de escoger lógicamente. Supongamos que tenemos 1000 fotos. El valor máximo para una foto es 255. Por lo tanto, necesitamos un tipo de datos que maneje un número de 1000 * 255 = 255,000. Entonces uint32 debería hacer el truco (0 a 4294967295)

In [ ]:
to float:
    temp = temp.astype('uint32')

A continuación, tenemos que 1) crear una nueva variable para contener la suma o 2) agregar la imagen actual a la imagen sumada

In [ ]:
    if first:
        sumImage = temp
        first = False
    else:
        sumImage = sumImage + temp

Ahora calculamos la imagen promediada dividiendo la imagen sumada por el número de imágenes:

In [ ]:
avgArray = sumImage/len(imgList)

Tenemos que volver a convertir al tipo de datos unit8, luego a la clase de imagen PIL:

In [ ]:
avgImg = Image.fromarray(avgArray.astype('uint8'))

El código resultaría de la siguiente forma:

In [25]:
from PIL import Image
import glob
import numpy as np

In [26]:
imgList = glob.glob('./ImageStAverage./*jpg')

In [27]:
print(type(imgList))
print(len(imgList))
print(imgList)

<class 'list'>
19
['./ImageStAverage.\\WIN_20190627_07_36_38_Pro (2).jpg', './ImageStAverage.\\WIN_20190627_07_36_38_Pro.jpg', './ImageStAverage.\\WIN_20190627_07_36_39_Pro (2).jpg', './ImageStAverage.\\WIN_20190627_07_36_39_Pro.jpg', './ImageStAverage.\\WIN_20190627_07_36_40_Pro (2).jpg', './ImageStAverage.\\WIN_20190627_07_36_40_Pro (3).jpg', './ImageStAverage.\\WIN_20190627_07_36_40_Pro.jpg', './ImageStAverage.\\WIN_20190627_07_36_41_Pro.jpg', './ImageStAverage.\\WIN_20190627_07_36_43_Pro (3).jpg', './ImageStAverage.\\WIN_20190627_07_36_43_Pro (4).jpg', './ImageStAverage.\\WIN_20190627_07_36_43_Pro.jpg', './ImageStAverage.\\WIN_20190627_07_36_44_Pro (2).jpg', './ImageStAverage.\\WIN_20190627_07_36_44_Pro (3).jpg', './ImageStAverage.\\WIN_20190627_07_36_44_Pro.jpg', './ImageStAverage.\\WIN_20190627_07_36_45_Pro (2).jpg', './ImageStAverage.\\WIN_20190627_07_36_45_Pro (3).jpg', './ImageStAverage.\\WIN_20190627_07_36_45_Pro (4).jpg', './ImageStAverage.\\WIN_20190627_07_36_45_Pro.jpg', '

In [28]:
def SumImg(dir):
    first = True
    for img in dir:
        temp = np.asarray(Image.open(img))
        temp = temp.astype('uint16')
        if first:
            sumImage = temp
            first = False
        else:
            sumImage = sumImage + temp
    return sumImage

In [29]:
suma=SumImg(imgList)
avgArray = suma/len(imgList)
avgImg = Image.fromarray(avgArray.astype('uint8'))
avgImg.show()#Se muestra la imagen